In [1]:
import pandas as pd
import numpy as np
import math
import random

In [2]:
# (2*(x-3)**2 + 4)
# (cos(2*x))


def f(x):
    return f"{(3*x + 4):.2f}"

def fdash(x):
    return 4*(x-3)

In [3]:
def convert(arr):
    arr = arr.T
    for j in range(len(arr)):
        x = arr[j]
        temp = []
        for i in x:
            temp.append(fdash(i))
        xdash = np.array(temp)
        temp = []
        for i in xdash:
            temp.append(f(i))
        arr[j] = np.array(temp)
    print(arr)
    arr = arr.T
    return arr


In [4]:
def rotationMatrix(axis,deg):
  thta = math.radians(deg)
  c, s = np.cos(thta), np.sin(thta)
  Rx = np.array([[1,0,0],[0,c,s],[0,-s,c]])
  Ry = np.array([[c,0,-s],[0,1,0],[s,0,c]])
  Rz = np.array([[c,-s,0],[s,c,0],[0,0,1]])
  RxRy = np.matrix(Rx)*np.matrix(Ry)
  RyRz = np.matrix(Ry)*np.matrix(Rz)
  RxRz = np.matrix(Rx)*np.matrix(Rz)
  if axis=="xy":
    R = RxRy
  elif axis=="yz":
    R = RyRz
  elif axis=="xz":
    R = RxRz
  return R  

In [5]:
def rotate(data,R):
  size = data.shape[1]
  reminder = size%3
  #print(size,reminder)

  #first triplate
  triplate = data[:,:3]
  merged_array = np.array(np.matrix(triplate)*np.matrix(R))
  for i in range(3,size-reminder,3):
    triplate = data[:,i:3+i]
    Rotated = np.array(np.matrix(triplate)*np.matrix(R))
    # Merge the arrays along the columns
    merged_array = np.concatenate((merged_array, Rotated), axis=1)

  #Rotate if any colums are left
  if reminder!=0:
    triplate = data[:,-3:]
    Rotated = np.array(np.matrix(triplate)*np.matrix(R))
    merged_array = np.concatenate((merged_array, Rotated[:,-1*(reminder):]), axis=1)
    #print(merged_array.shape)
  return merged_array

In [6]:
def _3drotation(D,S):
  n = D.shape[1]
  numOfTriplates = n//3
  triplatesList = []
  for i in range(0,numOfTriplates,3):
    triplatesList.append(D[:,i:i+3])
  triplatesList.append(D[:,-3:])
  triplates = np.array(triplatesList)
  Veq=0
  for q in ["xy", "yz", "xz"]:
    thetaq = [_ for _ in range(361)]
    thetatemp = thetaq
    for theta in thetaq:
      Rq = rotationMatrix(q,theta)
      i=0
      for triplate in triplates:
        dq = np.array(np.matrix(triplate)*np.matrix(Rq))
        v1 = (np.var(triplate[:,0]-dq[:,0]))
        v2 = (np.var(triplate[:,1]-dq[:,1]))
        v3 = (np.var(triplate[:,2]-dq[:,2]))
        if v1<S[i][0] or v2<S[i][1] or v3<S[i][2]:
          if theta in thetatemp:
            thetatemp.remove(theta)
        i+=1
    alpha = random.choice(thetatemp)
    Deq=rotate(D,rotationMatrix(q,alpha))
    V = np.var(Deq)
    #rint(f"Variance of {q} axis is {V} " )
    if Veq<V:
      Veq = V
      Ddash = Deq
  return Ddash

In [7]:
df = pd.read_csv('data.csv')
df.head()
label = df['Class']
df = df.drop(columns=['Class'])
A = df.to_numpy()
print(A.shape)
S = [[1,1,1],[.5,.5,.5],[.5,.5,.5],[.4,.4,.4],[.4,.4,.4],[1,1,1],[.5,.5,.5],[.5,.5,.5],[.4,.4,.4],[.4,.4,.4],[1,1,1],[.5,.5,.5],[.5,.5,.5],[.4,.4,.4],[.4,.4,.4],[1,1,1],[.5,.5,.5],[.5,.5,.5],[.4,.4,.4]]
newArr = _3drotation(convert(A.copy()),S)
print(newArr.shape)

df = pd.DataFrame(newArr)
df['Class'] = np.array(list(label))
df.to_csv("newData.csv",index=False)

(267, 44)
[[772. 868. 964. ... 772. 784. 760.]
 [784. 856. 736. ... 736. 748. 616.]
 [844. 820. 760. ... 844. 832. 796.]
 ...
 [856. 664. 736. ... 928. 640. 784.]
 [676. 652. 484. ... 772. 664. 676.]
 [784. 592. 472. ... 784. 496. 676.]]
(267, 44)
